In [2]:
import pandas as pd
import datetime
import numpy as np
from datetime import datetime,timedelta
import os
from timeit import default_timer as timer
from scipy import stats
from sklearn import linear_model
os.chdir('/Users/hudsonyeo/Desktop/Python/leo/data/day/TA')
file_list=os.listdir('/Users/hudsonyeo/Desktop/Python/leo/data/day/TA')
file_list.sort()

In [151]:
file_list

['.DS_Store',
 '2019.01.02.csv',
 '2019.01.03.csv',
 '2019.01.04.csv',
 '2019.01.07.csv',
 '2019.01.08.csv',
 '2019.01.09.csv',
 '2019.01.10.csv',
 '2019.01.11.csv',
 '2019.01.14.csv',
 '2019.01.15.csv',
 '2019.01.16.csv',
 '2019.01.17.csv',
 '2019.01.18.csv',
 '2019.01.21.csv',
 '2019.01.22.csv',
 '2019.01.23.csv',
 '2019.01.24.csv',
 '2019.01.25.csv',
 '2019.01.28.csv',
 '2019.01.29.csv',
 '2019.01.30.csv',
 '2019.01.31.csv',
 '2019.02.01.csv',
 '2019.02.11.csv',
 '2019.02.12.csv',
 '2019.02.13.csv',
 '2019.02.14.csv',
 '2019.02.15.csv',
 '2019.02.18.csv',
 '2019.02.19.csv',
 '2019.02.20.csv',
 '2019.02.21.csv',
 '2019.02.22.csv',
 '2019.02.25.csv',
 '2019.02.26.csv',
 '2019.02.27.csv',
 '2019.02.28.csv',
 '2019.03.01.csv',
 '2019.03.04.csv',
 '2019.03.05.csv',
 '2019.03.06.csv',
 '2019.03.07.csv',
 '2019.03.08.csv',
 '2019.03.11.csv',
 '2019.03.12.csv',
 '2019.03.13.csv',
 '2019.03.14.csv',
 '2019.03.15.csv',
 '2019.03.18.csv',
 '2019.03.19.csv',
 '2019.03.20.csv',
 '2019.03.21.csv'

In [152]:
class categorise():
    def __init__(self):
        self.threshold=[]
        self.percentiles=[25,50,75]
        
    def calc_thresholds(self,array): #get the percentile values of the array
        self.threshold.append(np.percentile(array,self.percentiles))   
     
    def return_quartile(self,array): 
        temp=[]
        for i in array:
                if i<self.threshold[0][0]:
                    temp.append(1)
                elif i<self.threshold[0][1]:
                    temp.append(2)
                elif i<self.threshold[0][2]:
                    temp.append(3)
                else:
                    temp.append(4)             
        return np.asarray(temp)
    
class categorise_sign():
    def __init__(self):
        self.threshold=[]
        self.percentiles=[25,50,75]
        
    def calc_thresholds(self,array):
        positive=array[array>0]
        negative=array[array<0]
        
        self.threshold.append(np.percentile(negative,self.percentiles))   
        self.threshold.append(np.percentile(positive,self.percentiles))
     
    def return_quartile(self,array):
        temp=[]
        for i in array:
            if i>=0:
                if i<self.threshold[1][0]:
                    temp.append(5)
                elif i<self.threshold[1][1]:
                    temp.append(6)
                elif i<self.threshold[1][2]:
                    temp.append(7)
                else:
                    temp.append(8)
            if i<0:
                if i>self.threshold[0][2]:
                    temp.append(4)
                elif i>self.threshold[0][1]:
                    temp.append(3)
                elif i>self.threshold[0][0]:
                    temp.append(2)
                else:
                    temp.append(1)
        return np.asarray(temp)
                
    
class categorise_vol():
    def __init__(self):
        self.threshold=[]
        self.percentiles=[10,25,50,75,90]
        
    def calc_thresholds(self,array): #get the percentile values of the array
        self.threshold.append(np.percentile(array,self.percentiles))   
     
    def return_quartile(self,array): 
        temp=[]
        for i in array:
                if i<self.threshold[0][0]:
                    temp.append(1)
                elif i<self.threshold[0][1]:
                    temp.append(2)
                elif i<self.threshold[0][2]:
                    temp.append(3)
                elif i<self.threshold[0][3]:
                    temp.append(4)
                elif i<self.threshold[0][4]:
                    temp.append(5)
                else:
                    temp.append(6)                    
        return np.asarray(temp) 
    
class LinearRegression(linear_model.LinearRegression):
    """
    LinearRegression class after sklearn's, but calculate t-statistics
    and p-values for model coefficients (betas).
    Additional attributes available after .fit()
    are `t` and `p` which are of the shape (y.shape[1], X.shape[1])
    which is (n_features, n_coefs)
    This class sets the intercept to 0 by default, since usually we include it
    in X.
    """
    def __init__(self, fit_intercept=True, normalize=False, copy_X=True,
                     n_jobs=1):
            self.fit_intercept = fit_intercept
            self.normalize = normalize
            self.copy_X = copy_X
            self.n_jobs = n_jobs
    def fit(self, X, y, n_jobs=1):
        self = super(LinearRegression, self).fit(X, y, n_jobs)

        sse = np.sum((self.predict(X) - y) ** 2, axis=0) / float(X.shape[0] - X.shape[1])
        se = np.array([
            np.sqrt(np.diagonal(sse[i] * np.linalg.inv(np.dot(X.T, X))))
                                                    for i in range(sse.shape[0])
                    ])

        self.t = self.coef_ / se
        self.p = 2 * (1 - stats.t.cdf(np.abs(self.t), y.shape[0] - X.shape[1]))
        return self    

In [154]:
def calc_smart_price(dataset):
    data=dataset[:]
    
    #to combat the limit up event, where price is set to 0. 
    rows=(data.loc[:,'BidPrice1']==0) #count rows of bid price equal 0
    if (np.any(rows)): #if there is such a row
        data.at[rows,'BidPrice1']=data.loc[rows,'AskPrice1'] #for that row, assign ask price to it
    rows=(data.loc[:,'AskPrice1']==0) #do the same for ask price
    if (np.any(rows)):
        data.at[rows,'AskPrice1']=data.loc[rows,'BidPrice1'] 
        
    data['smart_price']=data.loc[:,'BidPrice1']*data.loc[:,'AskVol1']+data.loc[:,'AskPrice1']*data.loc[:,'BidVol1']
    data.at[:,'smart_price']=data.loc[:,'smart_price']/(data.loc[:,['BidVol1','AskVol1']].sum(axis=1))  
    return data

def calc_present_vol(dataset):
    data=dataset[:]
    data['current_vol']=data.loc[:,'Volume'].diff().fillna(0)/2
    return data

def calc_future_price(dataset,time_ahead=30):
    data=dataset[:]
    future_price=[]
    length=len(data)
    for i in range(len(data)):
        current_time=data[i,44]+timedelta(seconds=time_ahead)
        #print(data[i,44])
        j=0
        #print(current_time)
        while((i+j)<length and current_time>data[(i+j),44]):
            j+=1
        #print(i,j,(data[(i+j-1),44]))
        if (i+j)<length:
            future_price.append(data[(i+j),51]) #51 is the index for smart price            
        else:
            future_price.append(np.nan)
    future_price=np.asarray(future_price)
    future_price=np.expand_dims(future_price,axis=1)
    return np.concatenate((data,future_price),axis=1)


def calc_edge(dataset):
    data=dataset.copy()
    temp=data[:,53]-data[:,51]
    temp=np.expand_dims(temp,axis=1)
    return np.concatenate((data,temp),axis=1)

def set_index(dataset):
    data=dataset[:]
    index=data[:,44]
    new_index=[]
    for j in range(len(index)):
        i=str(index[j]*1000)
        if len(i)==11:
            i='0'+i
        i=i[:-10]+':'+i[-10:]
        i=i[:-8]+':'+i[-8:]
        i=i[:-6]+':'+i[-6:]
        new_index.append(datetime.strptime(i,"%H:%M:%S:%f"))
    data[:,44]=new_index
    return data

def calc_sma_fast(dataset,duration=1): #0.05 seconds for 5000 rows
    data=dataset[:]
    sma_values=[] 
    smart_sum=np.cumsum(data[:,51])
    for i in range(len(data)):
        last_time=data[i,44]-timedelta(minutes=duration)
        j=220*duration#4x60=240
        while(i-j>0 and data[i-j,44]>last_time):
            j+=1
        if (i-j>=0):
            sma=(smart_sum[i]-smart_sum[i-j])/(j)
            sma_values.append(sma)
        else:
            sma=smart_sum[i]/(i+1)
            sma_values.append(sma)
    sma_values=np.asarray(sma_values)
    sma_values=data[:,51][:]-sma_values
    sma_values=np.expand_dims(sma_values,axis=1)
    return np.concatenate((data,sma_values),axis=1) 

def calc_past_vol(dataset,duration=1): #
    data=dataset[:]
    vol_values=[] 
    vol_sum=np.cumsum(data[:,52])
    for i in range(len(data)):
        last_time=data[i,44]-timedelta(minutes=duration)
        j=220*duration#4x60=240
        while(i-j>0 and data[i-j,44]>last_time):
            j+=1
        if (i-j>=0):
            vol=(vol_sum[i]-vol_sum[i-j])
            vol_values.append(vol)
        else:
            vol=vol_sum[i]
            vol_values.append(vol)
    vol_values=np.asarray(vol_values)
    vol_values=np.expand_dims(vol_values,axis=1)
    return np.concatenate((data,vol_values),axis=1) #52  

def calc_past_vol_seconds(dataset,duration=10): #
    data=dataset[:]
    vol_values=[] 
    vol_sum=np.cumsum(data[:,52])
    for i in range(len(data)):
        last_time=data[i,44]-timedelta(seconds=duration)
        j=2*duration#4 rows per second
        while(i-j>0 and data[i-j,44]>last_time):
            j+=1
        if (i-j>=0):
            vol=(vol_sum[i]-vol_sum[i-j])
            vol_values.append(vol)
        else:
            vol=vol_sum[i]
            vol_values.append(vol)
    vol_values=np.asarray(vol_values)
    vol_values=np.expand_dims(vol_values,axis=1)
    return np.concatenate((data,vol_values),axis=1) #52  

def process(dataset):
    data=dataset[:]
    data=calc_smart_price(data) #51
    data=calc_present_vol(data).values #52
    data=set_index(data)
    data=calc_future_price(data) #53
    data=calc_edge(data) #54
    data=calc_sma_fast(data,duration=1) #55
    data=calc_past_vol(data,duration=1) #56
    return data

ma_dict={'-4':'1',
        '-3':'5',
        '-2':'15',
        '-1':'30'}    
        
'''def calc_vwap(dataset,duration=1): #to be implement
    data=dataset[:]
    for i in data[:,44]:
        last_time=i-timedelta(minutes=duration)
        rolling=data[(data[:,44]>=last_time) & (data[:,44]<i)]
        high=rolling[:,51].max()
        low=rolling[:,51].min()
        avg=(rolling[-1,51]+high+low)/3
def calc_rsi(dataset)'''

'def calc_vwap(dataset,duration=1): #to be implement\n    data=dataset[:]\n    for i in data[:,44]:\n        last_time=i-timedelta(minutes=duration)\n        rolling=data[(data[:,44]>=last_time) & (data[:,44]<i)]\n        high=rolling[:,51].max()\n        low=rolling[:,51].min()\n        avg=(rolling[-1,51]+high+low)/3\ndef calc_rsi(dataset)'

In [ ]:
df_list=[]
name_list=[]
path='/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/'
for file in file_list: #read all files and add them to file_list
    if file[-3:]=='csv': #check if file is a CSV
        name_list.append(file)
        df_list.append(process(pd.read_csv(path+file)))
        print(file,'read')

In [157]:
#run regressions against all 4 moving averages
df_path='/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/results/'

final_df=pd.DataFrame()    

for i in range(len(df_list)): #for each 20 day rolling window
    if i<19:
        continue #skip the first 19 rows as we want a 20 day rolling period
    print(i,name_list[i])
        
    #get -19 day
    sma_past=df_list[i-19][:,55][:].copy() #55 is sma
    vol_past=df_list[i-19][:,56][:].copy() #56 is cumulative volume in past 1 minute
    #get -18 to 0 day (19 days in total)
    
    for k in range((i-18),(i+1)): #get 20 day moving averages
        sma_past=np.concatenate((sma_past,df_list[k][:,55].copy()))
        vol_past=np.concatenate((vol_past,df_list[k][:,56].copy()))
    #sma_past and vol_past are the past 20 days of sma and volume     

    #calculate quartile thresholds for past 20 days, for sma and vol
    cat_sma=categorise_sign()
    cat_vol=categorise_vol()
    cat_sma.calc_thresholds(sma_past) 
    cat_vol.calc_thresholds(vol_past)

    #get x,y for regression
    #55 is sma, 54 is edge, 56 is past 1 minute volume
    x=df_list[i][:,55][:].astype(float).copy() 
    y=df_list[i][:,54][:].astype(float).copy() 
    vol=df_list[i][:,56][:].astype(float).copy()
    
    #removing all NA
    isnum=(~np.isnan(x)) & (~np.isnan(y))
    y=y[isnum]
    x=x[isnum]        
    vol=vol[isnum]
    
    #get the categories for sma and vol
    category_sma=cat_sma.return_quartile(x)
    category_vol=cat_vol.return_quartile(vol)
    
    reg_result={}
    reg_result['date']=name_list[i]
    
    for sma_quartile in [1,2,3,4,5,6,7,8]:
        
        #filter by SMA first
        name='sma_'+str(sma_quartile)+'_vol'
        sma_filter=(category_sma==sma_quartile) #filter for SMA category
        reg_result['num_obs_'+str(sma_quartile)]=len(x[sma_filter]) #num of obs in Q1 or Q8
        
        for vol_quartile in [1,2,3,4,5,6]:

            q=name+str(vol_quartile)+'_'            
            #filter by quartile
            vol_filter=(category_vol==vol_quartile) #filter for volume category
            filtered=(sma_filter&vol_filter) #combine both filters using &
            
            if (len(x[filtered])>1):
                new_x=x[filtered] #using combined filters to get data for regression
                new_y=y[filtered] 
                new_x=np.expand_dims(new_x,1)
                new_y=np.expand_dims(new_y,1)
                lin_model=LinearRegression(fit_intercept=False)
                lin_model.fit(new_x,new_y)

                reg_result[(q+'slope')]=lin_model.coef_[0][0]
                reg_result[(q+'p_val')]=lin_model.p[0][0]
                reg_result[(q+'x_mean')]=np.mean(new_x)
                reg_result[(q+'x_std')]=np.std(new_x)
                reg_result[(q+'num_obs')]=len(new_x)
            else:
                reg_result[(q+'slope')]='na'
                reg_result[(q+'p_val')]='na'
                reg_result[(q+'x_mean')]='na'
                reg_result[(q+'x_std')]='na'
                reg_result[(q+'num_obs')]='0'
    final_df=final_df.append(reg_result,ignore_index=True)                
temp=df_path+'result_Q1_to_8_1minSMA_6split_volume_nointercept.csv'
final_df.to_csv(temp)
print('done',temp)

19 2019.01.29.csv
20 2019.01.30.csv
21 2019.01.31.csv
22 2019.02.01.csv
23 2019.02.11.csv
24 2019.02.12.csv
25 2019.02.13.csv
26 2019.02.14.csv
27 2019.02.15.csv
28 2019.02.18.csv
29 2019.02.19.csv
30 2019.02.20.csv
31 2019.02.21.csv
32 2019.02.22.csv
33 2019.02.25.csv
34 2019.02.26.csv
35 2019.02.27.csv
36 2019.02.28.csv
37 2019.03.01.csv
38 2019.03.04.csv
39 2019.03.05.csv
40 2019.03.06.csv
41 2019.03.07.csv
42 2019.03.08.csv
43 2019.03.11.csv
44 2019.03.12.csv
45 2019.03.13.csv
46 2019.03.14.csv
47 2019.03.15.csv
48 2019.03.18.csv
49 2019.03.19.csv
50 2019.03.20.csv
51 2019.03.21.csv
52 2019.03.22.csv
53 2019.03.25.csv
54 2019.03.26.csv
55 2019.03.27.csv
56 2019.03.28.csv
57 2019.03.29.csv
58 2019.04.01.csv
59 2019.04.02.csv
60 2019.04.03.csv
61 2019.04.04.csv
62 2019.04.08.csv
63 2019.04.09.csv
64 2019.04.10.csv
65 2019.04.11.csv
66 2019.04.12.csv
67 2019.04.15.csv
68 2019.04.16.csv
69 2019.04.17.csv
70 2019.04.18.csv
71 2019.04.19.csv
72 2019.04.22.csv
73 2019.04.23.csv
74 2019.04

In [3]:
data=pd.read_csv('/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/results/result_Q1_to_8_1minSMA_6split_volume_nointercept.csv')
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
for i in [1,2,3,4,5,6,7,8]:
    for j in range(1,7):
        name='sma_'+str(i)+'_vol'+str(j)+'_slope'
        series=data.loc[:,name]
        x=[]
        for k in data.loc[:,name]:
            if is_number(k):
                if k==k:
                    x.append(float(k))
        x=np.asarray(x)
        pos=x[x>0]
        print(name,'percent positive',round(len(pos)/len(x),5),'mean',round(np.mean(x),5),'std',round(np.std(x),5))
        
        

sma_1_vol1_slope percent positive 0.57143 mean 0.07873 std 0.58129
sma_1_vol2_slope percent positive 0.5 mean 0.03297 std 0.28254
sma_1_vol3_slope percent positive 0.51562 mean 0.02332 std 0.19355
sma_1_vol4_slope percent positive 0.51562 mean 0.01266 std 0.16475
sma_1_vol5_slope percent positive 0.53125 mean 0.02083 std 0.18885
sma_1_vol6_slope percent positive 0.40323 mean -0.07593 std 0.22411
sma_2_vol1_slope percent positive 0.73438 mean 0.31949 std 0.73591
sma_2_vol2_slope percent positive 0.64062 mean 0.10861 std 0.34986
sma_2_vol3_slope percent positive 0.625 mean 0.06296 std 0.20833
sma_2_vol4_slope percent positive 0.65625 mean 0.11192 std 0.2476
sma_2_vol5_slope percent positive 0.375 mean -0.08097 std 0.42557
sma_2_vol6_slope percent positive 0.50794 mean 0.02843 std 0.6747
sma_3_vol1_slope percent positive 0.73438 mean 0.309 std 0.76822
sma_3_vol2_slope percent positive 0.64062 mean 0.2165 std 0.44379
sma_3_vol3_slope percent positive 0.6875 mean 0.17881 std 0.41153
sma_3_v